Extension

# Dataset
## Fichier d'instance de problème

## Valeur de l'optimal ou borne inférieure

## Code de génération aléatoire

# Code Python de l'algorithme


# Codes pour générer l'étude expérimentale

## Code de calcul des paramètres des graphes et des bornes inférieures

## Code de calcul des statistiques descriptives, voire prédictives

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats as st
import matplotlib.pyplot as plt

def main(argv):
    # On recupere le Dataset
    with open("Dataset/Data.json".format(argv[0]), 'r') as json_file:
        data = json.load(json_file)


    datademand = data['demands']
    datavehicule = data['vehicle_capacities']
    
if __name__ == '__main__':
    main(sys.argv[1:])


NameError: name 'argv' is not defined

### Moyenne

### Médiane

### Quantiles

### Variance

### L'écart-type

### L'écart interquartile

## Script de tests de montée en charge